In [1]:
import os
import numpy as np
import flask
import pickle
from flask import Flask, render_template, request, Response, jsonify
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
import tensorflow as tf
#from util import base64_to_pil

from PIL import Image
from tensorflow.keras.preprocessing import image

In [ ]:
# Declare a flask app
app = Flask(__name__)


#print('Model loaded. Check http://127.0.0.1:5000/')

loaded_model = pickle.load(open("cnn_model.pkl", "rb"))

print('Model loaded. Start serving...')


#MODEL_PATH = 'models/oldModel.h5'

#model = load_model(MODEL_PATH)


def model_predict(img_path):
    
    img = image.load_img(img_path, target_size=(120,120))
    
    img_array = tf.keras.utils.img_to_array(img)
    img_generator = ImageDataGenerator(rescale = 1./255)
    img_array = img_generator.standardize(img_array)

    #img_array = tf.expand_dims(img_array, 0) # Create a batch
    img_array = img_array.reshape(1 , 120 ,120 ,3)

    predictions = loaded_model.predict(img_array)
    #result = loaded_model.predict(img_array)

    #result = predictions[0,0]
    
    return predictions

# routes
@app.route("/", methods=['GET', 'POST'])
def main():
    return render_template("index.html")

@app.route("/result2", methods = ['GET', 'POST'])
def get_output():
    if request.method == 'POST':
        dummy = 'success'
        #img = request.files['my_image']
        img = request.files['file']

        img_path = "static/" + img.filename
        img.save(img_path)

        p = model_predict(img_path)
        
        result = p[0][0]
        
        if result >0.5:
            #return jsonify(result="PNEMONIA")
            prediction = 'PNEMONIA'
        else:
            #return jsonify(result="NORMAL")
            prediction = 'Normal'

    #return render_template("result2.html", prediction = dummy)
    return render_template("result2.html", prediction = prediction, img_path = img_path)
    

if __name__ == '__main__':
    app.run()


Model loaded. Start serving...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/May/2024 04:55:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 04:55:47] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step


127.0.0.1 - - [16/May/2024 04:57:02] "POST /result2 HTTP/1.1" 200 -
